In [2]:
import pandas as pd

df = pd.read_csv("movie_train.csv",index_col=0,)

df.reset_index(drop=False,inplace=True)
df.rename(mapper={'index':'ID'},axis=1,inplace=True)

print(df.shape)

df.head()




(10682, 7)


,index,Release Year,Title,Plot,Director,Cast,Genre
0,10281,1984,Silent Madness,A computer error leads to the accidental relea...,Simon Nuchtern,"Belinda Montgomery, Viveca Lindfors",horror
1,7341,1960,Desire in the Dust,"Lonnie Wilson (Ken Scott), the son of a sharec...",Robert L. Lippert,"Raymond Burr, Martha Hyer, Joan Bennett",drama
2,10587,1986,On the Edge,"A gaunt, bushy-bearded, 44-year-old Wes Holman...",Rob Nilsson,"Bruce Dern, Pam Grier",drama
3,25495,1988,Ram-Avtar,Ram and Avtar are both childhood best friends....,Sunil Hingorani,"Sunny Deol, Anil Kapoor, Sridevi",drama
4,16607,2013,Machete Kills,Machete Cortez (Danny Trejo) and Sartana River...,Robert Rodriguez,"Danny Trejo, Michelle Rodriguez, Sofía Vergara...",action


In [4]:
df['index'].duplicated().sum()

0

In [7]:
df.rename(mapper={'index':'ID'},axis=1,inplace=True)

In [10]:
def CreateCorpusFromDataFrame(corpusfolder,df):
    for index, r in df.iterrows():
        id=r['ID']
        release_year=r['Release Year']
        title=r['Title']
        plot=r['Plot']
        director=r['Director']
        cast=r['Cast']
        genre=r['Genre']
        fname=str(genre)+'_'+str(id)+'.txt'
        corpusfile=open(corpusfolder+'/'+fname,'a')
        corpusfile.write(str(plot) +" " +str(title))
        corpusfile.close()

# CreateCorpusFromDataFrame('yourcorpusfolder/',df)

In [11]:
corpus_folder = "/Users/schlinkertc/Flatiron/Lecture_Notes/nyc-ds-100719-lectures/week-11/NLP/classification-assessment/corpus"

CreateCorpusFromDataFrame(corpus_folder,df)

In [12]:
from nltk.corpus.reader import CategorizedPlaintextCorpusReader

my_corpus=CategorizedPlaintextCorpusReader(corpus_folder,
r'.*', cat_pattern=r'(.*)_.*')

In [62]:
my_corpus.fileids() # <- I expect values from column ID
my_corpus.categories() # <- I expect values from column CATEGORY
#my_corpus.sents(categories='action') # <- I expect values from column TITLE and BODY
#my_corpus.sents(categories=['cat_A', 'cat_B']) # <- I expect values from column TITLE and BODY

['action',
 'adventure',
 'comedy',
 'crime',
 'drama',
 'horror',
 'romance',
 'thriller',
 'western']

In [101]:
## attempt to build custom corpus reader class
from nltk.corpus.reader.api import CorpusReader
from nltk.corpus.reader.api import CategorizedCorpusReader
import codecs
import os 

CAT_PATTERN = r'(.*)_.*'
DOC_PATTERN = r'.*'

class my_CorpusReader(CategorizedCorpusReader,CorpusReader):
    """
    A corpus reader to enable preprocessing
    """
    
    def __init__(self, root, fileids=DOC_PATTERN, encoding='utf8',**kwargs):
        
        """
        Initialize the corpus reader.
        """
        
        # add the default category pattern if not passed into the class.
        if not any(key.startswith('cat_') for key in kwargs.keys()):
            kwargs['cat_pattern'] = CAT_PATTERN
            
        # Initialize the NLTK corpus reader objects
        CategorizedCorpusReader.__init__(self,kwargs)
        CorpusReader.__init__(self, root, fileids, encoding)
        
    def resolve(self, fileids, categories):
        """
        Returns a list of fileids or categories depending on what is passed
        """

        if fileids is not None and categories is not None:
            raise ValueError("specify either files or cat, not both")

        if categories is not None:
            return self.fileids(categories)
        return fileids

    def docs(self, fileids, categories=None):
        """
        Returns the complete text of a document, closing the doc 
        after we're done reading it and yielding it in a memory-safe fashion.
        """

        # Resolve the fileids and the categories
        fileids = self.resolve(fileids, categories)

        # Create a generator, loading one doc into memory at a time
        for path, encoding in self.abspaths(fileids,include_encoding=True):
            with codecs.open(path, 'r', encoding=encoding) as f:
                yield f.read()

    def sizes(self, fileids=None, categories=None):
        """
        Returns a list of tuples, the fileid and size on disk of the file.
        This function is used to detect oddly large files in the corpus.
        """
        # Resolve the fileids and the categories
        fileids = self.resolve(fileids, categories)

        # Create a generator, getting every path and computing filesize
        for path in self.abspaths(fileids):
            yield os.path.getsize(path)


In [140]:
from nltk import pos_tag, sent_tokenize, wordpunct_tokenize
class Preprocessor(object):
    """
    The preprocessor wraps a corpus object (usually a `HTMLCorpusReader`)
    and manages the stateful tokenization and part of speech tagging into a
    directory that is stored in a format that can be read by the
    `HTMLPickledCorpusReader`. This format is more compact and necessarily
    removes a variety of fields from the document that are stored in the JSON
    representation dumped from the Mongo database. This format however is more
    easily accessed for common parsing activity.
    """

    def __init__(self, corpus, target=None, **kwargs):
        """
        The corpus is the `HTMLCorpusReader` to preprocess and pickle.
        The target is the directory on disk to output the pickled corpus to.
        """
        self.corpus = corpus
        self.target = target

    def fileids(self, fileids=None, categories=None):
        """
        Helper function access the fileids of the corpus
        """
        fileids = self.corpus.resolve(fileids, categories)
        if fileids:
            return fileids
        return self.corpus.fileids()

    def abspath(self, fileid):
        """
        Returns the absolute path to the target fileid from the corpus fileid.
        """
        # Find the directory, relative from the corpus root.
        parent = os.path.relpath(
            os.path.dirname(self.corpus.abspath(fileid)), self.corpus.root
        )

        # Compute the name parts to reconstruct
        basename  = os.path.basename(fileid)
        name, ext = os.path.splitext(basename)

        # Create the pickle file extension
        basename  = name + '.pickle'

        # Return the path to the file relative to the target.
        return os.path.normpath(os.path.join(self.target, parent, basename))

    def tokenize(self, fileid):
        """
        Segments, tokenizes, and tags a document in the corpus. Returns a
        generator of paragraphs, which are lists of sentences, which in turn
        are lists of part of speech tagged words.
        """
        for document in self.corpus.docs(fileids=fileid):
            yield [
                pos_tag(wordpunct_tokenize(sent))
                for sent in sent_tokenize(document)
            ]

    def process(self, fileid):
        """
        For a single file does the following preprocessing work:
            1. Checks the location on disk to make sure no errors occur.
            2. Gets all paragraphs for the given text.
            3. Segements the paragraphs with the sent_tokenizer
            4. Tokenizes the sentences with the wordpunct_tokenizer
            5. Tags the sentences using the default pos_tagger
            6. Writes the document as a pickle to the target location.
        This method is called multiple times from the transform runner.
        """
        # Compute the outpath to write the file to.
        target = self.abspath(fileid)
        parent = os.path.dirname(target)

        # Make sure the directory exists
        if not os.path.exists(parent):
            os.makedirs(parent)

        # Make sure that the parent is a directory and not a file
        if not os.path.isdir(parent):
            raise ValueError(
                "Please supply a directory to write preprocessed data to."
            )

        # Create a data structure for the pickle
        document = list(self.tokenize(fileid))

        # Open and serialize the pickle to disk
        with open(target, 'wb') as f:
            pickle.dump(document, f, pickle.HIGHEST_PROTOCOL)

        # Clean up the document
        del document

        # Return the target fileid
        return target

    def transform(self, fileids=None, categories=None):
        """
        Transform the wrapped corpus, writing out the segmented, tokenized,
        and part of speech tagged corpus as a pickle to the target directory.
        This method will also directly copy files that are in the corpus.root
        directory that are not matched by the corpus.fileids().
        """
        # Make the target directory if it doesn't already exist
        if not os.path.exists(self.target):
            os.makedirs(self.target)

        # Resolve the fileids to start processing and return the list of 
        # target file ids to pass to downstream transformers. 
        return [
            self.process(fileid)
            for fileid in self.fileids(fileids, categories)
        ]

In [204]:
import time
import nltk
import pickle
from nltk.corpus.reader.api import CorpusReader
from nltk.corpus.reader.api import CategorizedCorpusReader

CAT_PATTERN = r'(.*)_.*'
DOC_PATTERN = r'.*'
PKL_PATTERN = r'.*'

class PickledCorpusReader(my_CorpusReader):

    def __init__(self, root, fileids=PKL_PATTERN, **kwargs):
        """
        Initialize the corpus reader.  Categorization arguments
        (``cat_pattern``, ``cat_map``, and ``cat_file``) are passed to
        the ``CategorizedCorpusReader`` constructor.  The remaining arguments
        are passed to the ``CorpusReader`` constructor.
        """
        # Add the default category pattern if not passed into the class.
        if not any(key.startswith('cat_') for key in kwargs.keys()):
            kwargs['cat_pattern'] = CAT_PATTERN

        CategorizedCorpusReader.__init__(self, kwargs)
        CorpusReader.__init__(self, root, fileids)

    def resolve(self, fileids, categories):
        """
        Returns a list of fileids or categories depending on what is passed
        to each internal corpus reader function. This primarily bubbles up to
        the high level ``docs`` method, but is implemented here similar to
        the nltk ``CategorizedPlaintextCorpusReader``.
        """
        if fileids is not None and categories is not None:
            raise ValueError("Specify fileids or categories, not both")

        if categories is not None:
            return self.fileids(categories)
        return fileids

    def docs(self, fileids=None, categories=None):
        """
        Returns the document loaded from a pickled object for every file in
        the corpus. Similar to the BaleenCorpusReader, this uses a generator
        to acheive memory safe iteration.
        """
        # Resolve the fileids and the categories
        fileids = self.resolve(fileids, categories)

        # Create a generator, loading one document into memory at a time.
        for path, enc, fileid in self.abspaths(fileids, True, True):
            with open(path, 'rb') as f:
                yield pickle.load(f)

    def paras(self, fileids=None, categories=None):
        """
        Returns a generator of paragraphs where each paragraph is a list of
        sentences, which is in turn a list of (token, tag) tuples.
        """
        for doc in self.docs(fileids, categories):
            for paragraph in doc:
                yield paragraph

    def sents(self, fileids=None, categories=None):
        """
        Returns a generator of sentences where each sentence is a list of
        (token, tag) tuples.
        """
        for paragraph in self.paras(fileids, categories):
            for sentence in paragraph:
                yield sentence

    def tagged(self, fileids=None, categories=None):
        for sent in self.sents(fileids, categories):
            for token in sent:
                yield token

    def words(self, fileids=None, categories=None):
        """
        Returns a generator of (token, tag) tuples.
        """
        for token in self.tagged(fileids, categories):
            yield token[0]

In [181]:
corpus = my_CorpusReader(corpus_folder)

In [182]:
target = "/Users/schlinkertc/Flatiron/Lecture_Notes/nyc-ds-100719-lectures/week-11/NLP/classification-assessment/processed_corpus"

In [183]:
processed_corpus = Preprocessor(corpus,target=target)

In [184]:
processed_corpus.transform()

['/Users/schlinkertc/Flatiron/Lecture_Notes/nyc-ds-100719-lectures/week-11/NLP/classification-assessment/processed_corpus/action_10011.pickle',
 '/Users/schlinkertc/Flatiron/Lecture_Notes/nyc-ds-100719-lectures/week-11/NLP/classification-assessment/processed_corpus/action_10055.pickle',
 '/Users/schlinkertc/Flatiron/Lecture_Notes/nyc-ds-100719-lectures/week-11/NLP/classification-assessment/processed_corpus/action_10095.pickle',
 '/Users/schlinkertc/Flatiron/Lecture_Notes/nyc-ds-100719-lectures/week-11/NLP/classification-assessment/processed_corpus/action_10107.pickle',
 '/Users/schlinkertc/Flatiron/Lecture_Notes/nyc-ds-100719-lectures/week-11/NLP/classification-assessment/processed_corpus/action_10121.pickle',
 '/Users/schlinkertc/Flatiron/Lecture_Notes/nyc-ds-100719-lectures/week-11/NLP/classification-assessment/processed_corpus/action_10156.pickle',
 '/Users/schlinkertc/Flatiron/Lecture_Notes/nyc-ds-100719-lectures/week-11/NLP/classification-assessment/processed_corpus/action_10157.p

In [343]:
import numpy as np

from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split as tts

class CorpusLoader(object):

    def __init__(self, reader, folds=12, shuffle=True, categories=None):
        self.reader = reader
        self.folds  = KFold(n_splits=folds, shuffle=shuffle)
        self.files  = np.asarray(self.reader.fileids(categories=categories))

    def fileids(self, idx=None):
        if idx is None:
            return self.files
        return self.files[idx]

    def documents(self, idx=None):
        for fileid in self.fileids(idx):
            return list(self.reader.docs(fileids=[fileid]))

    def labels(self, idx=None):
        return [
            self.reader.categories(fileids=[fileid])[0]
            for fileid in self.fileids(idx)
        ]

    def __iter__(self):
        for train_index, test_index in self.folds.split(self.files):
            X_train = self.documents(train_index)
            y_train = self.labels(train_index)

            X_test = self.documents(test_index)
            y_test = self.labels(test_index)

            yield X_train, X_test, y_train, y_test

In [344]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline

def identity(words):
    return words

In [345]:
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.base import BaseEstimator, TransformerMixin
from gensim.matutils import sparse2full

class TextNormalizer(BaseEstimator, TransformerMixin):

    def __init__(self, language='english'):
        self.stopwords  = set(nltk.corpus.stopwords.words(language))
        self.lemmatizer = WordNetLemmatizer()

    def is_punct(self, token):
        return all(
            unicodedata.category(char).startswith('P') for char in token
        )

    def is_stopword(self, token):
        return token.lower() in self.stopwords

    def normalize(self, document):
        return [
            self.lemmatize(token).lower()
            for paragraph in document
            for sentence in paragraph
            for token in sentence
            if not self.is_punct(token) and not self.is_stopword(token)
        ]

    def lemmatize(self, token):
        tag = {
            'N': wn.NOUN,
            'V': wn.VERB,
            'R': wn.ADV,
            'J': wn.ADJ
        }.get(pos_tag, wn.NOUN)

        return self.lemmatizer.lemmatize(token)

    def fit(self, X, y=None):
        return self

    def transform(self, documents):
        for document in documents:
            yield self.normalize(document[0])

In [346]:
def create_pipeline(estimator, reduction=False):

    steps = [
        ('normalize', TextNormalizer()),
        ('vectorize', TfidfVectorizer(
            tokenizer=identity, preprocessor=None, lowercase=False
        ))
    ]

    if reduction:
        steps.append((
            'reduction', TruncatedSVD(n_components=100)
        ))

    # Add the estimator
    steps.append(('classifier', estimator))
    return Pipeline(steps)

In [347]:
labels = my_corpus.categories()
reader = PickledCorpusReader(target)
loader = CorpusLoader(reader, 5, shuffle=True, categories=labels)

In [348]:
models = []
for form in (LogisticRegression, SGDClassifier):
    models.append(create_pipeline(form(), True))
    models.append(create_pipeline(form(), False))

models.append(create_pipeline(MultinomialNB(), False))
models.append(create_pipeline(GaussianNB(), True))

In [349]:
# from sklearn.metrics import f1_score

# for model in models:
#     scores = []
    
#     for X_train, X_test, y_train, y_test in loader:
#         model.fit(X_train,y_train)
#         y_pred = model.predict(X_test)
        
#         score = f1_score(y_test,y_pred)
#         scores.append(score)
        
#     print("f1 of {} is {:0.3f}".format(model, np.mean(scores)))

In [359]:
test_model = models[1]

In [360]:
test_model.get_params

<bound method Pipeline.get_params of Pipeline(memory=None,
         steps=[('normalize', TextNormalizer(language=None)),
                ('vectorize',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=False, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, stri...
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function identity at 0x1a91e91158>,
                                 use_idf=True, vocabulary=None)),
                ('classifier',
                 LogisticRegression(C=1.0, class_weight=None, dual=Fal

In [361]:
trains = []
for X_train, X_test, y_train, y_test in loader:
    trains.append(X_train)

In [366]:
scores = []
for X_train, X_test, y_train, y_test in loader:
    test_model.fit(X_train,y_train)
    y_pred = test_model.predict(X_test)
    scores.append(f1_score(y_train,y_pred))

//anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


ValueError: Found input variables with inconsistent numbers of samples: [1, 8545]

In [249]:
test = []
for X_train, X_test, y_train, y_test in loader:
    test.append(X_train)